In [1]:
import pandas as pd 
import datetime
from shapely import wkt, Point
from tqdm import tqdm

<h3>For regular staypoints.csv</h3>

In [2]:
tripleg = pd.read_csv('triplegs.csv')
stop_df = pd.read_csv('staypoints.csv')

In [3]:
stop_df

,id,user_id,started_at,finished_at,geom
0,0,0,2022-10-10 18:41:33-04:00,2022-10-10 18:54:04-04:00,POINT (-81.6625549999999976 30.4599150000000023)
1,1,2,2022-09-06 22:09:29-04:00,2022-09-06 22:19:08-04:00,POINT (-81.6582657950000055 29.6534941799999991)
2,2,2,2022-09-06 22:19:08-04:00,2022-09-06 22:22:07-04:00,POINT (-81.6840639999999922 29.6517550000000014)
3,3,2,2022-09-06 22:22:07-04:00,2022-09-06 22:31:37-04:00,POINT (-81.6582655899999992 29.6534943599999998)
4,4,2,2022-09-06 22:31:37-04:00,2022-09-06 22:34:49-04:00,POINT (-81.6840650000000181 29.6517550000000014)
...,...,...,...,...,...
397678,397678,34483,2022-10-27 03:34:53-04:00,2022-10-27 03:37:40-04:00,POINT (-81.6483199999999982 30.3217199999999991)
397679,397679,34483,2022-10-27 05:30:52-04:00,2022-10-27 05:38:14-04:00,POINT (-81.6525199999999955 30.3216099999999997)
397680,397680,34483,2022-10-27 05:38:14-04:00,2022-10-27 05:53:02-04:00,POINT (-81.6508199999999960 30.3522199999999991)
397681,397681,34484,2022-10-28 00:47:04-04:00,2022-10-28 00:55:00-04:00,POINT (-81.6688213850000011 30.3685913700000008)


<h3>The end location and time is start time of next stop point.
</h3>

In [4]:
stop_df['started_at'] = pd.to_datetime(stop_df['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [5]:
len(stop_df)

397683

In [6]:
ID_list=stop_df['user_id'].unique()
len(ID_list)

10987

In [7]:
time_window = datetime.timedelta(minutes=5)

miss = 0
check_len = 0
data_to_append_list = []
for id in tqdm(ID_list):
    stop_by_user = stop_df[stop_df['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_row in stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_row[3] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_row[3])
        ]
        if len(tripled_found) == 0: continue #did not found related trip
        #print(tripled_found)
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        '''
            The end location and time is start time of next stop point.
        '''
        
        data_to_append = {'deviceID': id, 'tripleg_ID': tripled_found.iloc[0]['id'], 
                          'trip_start_location': Point(list(wkt.loads(tripled_found.iloc[0]['geom']).coords)[0]),
                          'trip_end_location': stop_row[5],
                          'trip_start_timestamp': tripled_found.iloc[0]['started_at'],
                          'trip_end_timestamp': stop_row[3]}
        
        data_to_append_list.append(data_to_append)
        

trip_df = pd.DataFrame(data_to_append_list, columns=['deviceID', 'tripleg_ID', 'trip_start_location', 'trip_end_location', 'trip_start_timestamp', 'trip_end_timestamp'])

100%|██████████| 10987/10987 [03:46<00:00, 48.53it/s] 


In [8]:
trip_df

,deviceID,tripleg_ID,trip_start_location,trip_end_location,trip_start_timestamp,trip_end_timestamp
0,8,387,POINT (-81.79823 30.05037),POINT (-81.6317009899999988 30.1679205900000014),2022-09-23 12:15:13-04:00,2022-09-23 12:16:44-04:00
1,10,799,POINT (-81.650498 30.4701),POINT (-81.6504979099999986 30.4700875300000007),2022-10-07 22:00:05-04:00,2022-10-07 22:00:50-04:00
2,10,805,POINT (-81.63414717 30.47832727),POINT (-81.6381756050000007 30.4742551599999985),2022-10-08 13:45:17-04:00,2022-10-08 13:45:39-04:00
3,10,811,POINT (-81.62109 30.47463),POINT (-81.6363910000000175 30.4794150000000030),2022-10-09 13:00:02-04:00,2022-10-09 13:01:38-04:00
4,10,838,POINT (-81.6505 30.4701),POINT (-81.6502799999999951 30.4801349999999971),2022-10-11 22:00:32-04:00,2022-10-11 22:01:34-04:00
...,...,...,...,...,...,...
49291,34471,943523,POINT (-82.5793575 29.9798013),POINT (-82.5527774999999906 29.9126556000000008),2022-10-25 20:25:42-04:00,2022-10-25 20:29:10-04:00
49292,34471,943525,POINT (-82.78530836 30.29825449),POINT (-82.8365240499999942 30.3513787500000021),2022-10-30 01:52:42-04:00,2022-10-30 01:55:38-04:00
49293,34471,943526,POINT (-82.98503 30.48014),POINT (-83.0120300600000007 30.4881524450000008),2022-10-30 02:08:36-04:00,2022-10-30 02:10:02-04:00
49294,34471,943526,POINT (-82.98503 30.48014),POINT (-83.0472241500000052 30.5070582500000000),2022-10-30 02:08:36-04:00,2022-10-30 02:12:13-04:00


In [9]:
trip_df.to_csv('trip_5.csv')

<h3>For retail stop</h3>

In [10]:
stop_df = pd.read_csv('200m_food_related_stop.csv')
tripleg = pd.read_csv('triplegs.csv')

In [11]:
stop_df

,Unnamed: 0,id,user_id,started_at,finished_at,lat,lon,retail_id,retail_lat,retail_lon
0,0,8,2,2022-09-10 02:09:19-04:00,2022-09-10 02:11:50-04:00,29.655040,-81.657860,145696,29.655165,-81.656845
1,1,16,2,2022-09-26 02:07:17-04:00,2022-09-26 02:19:51-04:00,29.655035,-81.657880,145696,29.655165,-81.656845
2,2,18,3,2022-09-05 20:01:05-04:00,2022-09-05 20:04:13-04:00,29.651755,-82.339455,70229,29.653463,-82.339195
3,3,20,3,2022-09-07 20:02:10-04:00,2022-09-07 20:06:00-04:00,29.651742,-82.339464,70229,29.653463,-82.339195
4,4,22,3,2022-09-08 20:01:10-04:00,2022-09-08 20:04:18-04:00,29.651755,-82.339455,70229,29.653463,-82.339195
...,...,...,...,...,...,...,...,...,...,...
54533,54533,397640,34471,2022-10-22 02:50:59-04:00,2022-10-22 03:00:35-04:00,29.407710,-82.248080,18330,29.406757,-82.248161
54534,54534,397668,34471,2022-10-30 01:31:03-04:00,2022-10-30 01:33:29-04:00,30.002802,-82.597425,30767,30.001783,-82.597435
54535,54535,397674,34472,2022-10-26 23:04:35-04:00,2022-10-26 23:07:15-04:00,30.322587,-81.557411,58188,30.321955,-81.556618
54536,54536,397675,34481,2022-10-25 20:52:41-04:00,2022-10-25 20:56:31-04:00,29.651005,-82.330360,52511,29.650291,-82.330154


In [12]:
tripleg

,id,user_id,started_at,finished_at,geom
0,1,0,2022-09-25 22:02:49-04:00,2022-09-25 22:03:38-04:00,LINESTRING (-81.6623854599999959 30.4500031500...
1,2,1,2022-09-06 13:11:03-04:00,2022-09-06 13:11:54-04:00,LINESTRING (-82.0707699999999960 29.1930100000...
2,6,1,2022-09-09 00:59:39-04:00,2022-09-09 00:59:49-04:00,LINESTRING (-82.1166076699999934 29.1995639800...
3,7,2,2022-09-06 09:15:08-04:00,2022-09-06 09:15:58-04:00,LINESTRING (-81.6775345799999997 29.6550393099...
4,8,2,2022-09-06 21:34:07-04:00,2022-09-06 21:34:57-04:00,LINESTRING (-81.6840577100000047 29.6517777400...
...,...,...,...,...,...
896148,943571,34494,2022-10-27 01:06:08-04:00,2022-10-27 01:09:53-04:00,LINESTRING (-81.6119800000000026 30.3824900000...
896149,943572,34496,2022-10-31 02:06:17-04:00,2022-10-31 02:07:50-04:00,LINESTRING (-81.7572399999999959 30.3360599999...
896150,943573,34497,2022-10-26 08:25:34-04:00,2022-10-26 08:25:48-04:00,LINESTRING (-82.0980799999999959 29.9914400000...
896151,943574,34497,2022-10-29 20:39:33-04:00,2022-10-29 20:39:40-04:00,LINESTRING (-82.2332099999999997 29.3812399999...


In [13]:
stop_df['started_at'] = pd.to_datetime(stop_df['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])
ID_list=stop_df['user_id'].unique()
len(ID_list)

4286

In [14]:
time_window = datetime.timedelta(minutes=15)

miss = 0
check_len = 0
data_to_append_list = []
for id in tqdm(ID_list):
    stop_by_user = stop_df[stop_df['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_row in stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_row[4] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_row[4])
        ]
        if len(tripled_found) == 0: continue #did not found related trip
        #print(tripled_found)
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        '''
            The end location and time is start time of next stop point.
        '''
        
        data_to_append = {'deviceID': id, 'tripleg_ID': tripled_found.iloc[0]['id'], 
                          'trip_start_location': Point(list(wkt.loads(tripled_found.iloc[0]['geom']).coords)[0]),
                          'trip_end_location': (stop_row[6], stop_row[7]),
                          'trip_start_timestamp': tripled_found.iloc[0]['started_at'],
                          'trip_end_timestamp': stop_row[4]}
        
        data_to_append_list.append(data_to_append)
        

trip_df = pd.DataFrame(data_to_append_list, columns=['deviceID', 'tripleg_ID', 'trip_start_location', 'trip_end_location', 'trip_start_timestamp', 'trip_end_timestamp'])

100%|██████████| 4286/4286 [00:40<00:00, 106.72it/s]


In [15]:
print(len(trip_df), len(stop_df))

10378 54538


In [16]:
trip_df.to_csv('trip_food_relate_15.csv')